In [1]:
import sys
from pathlib import Path

CODE_ROOT = Path().resolve().parent   
sys.path.insert(0, str(CODE_ROOT))


In [2]:
import numpy as np
import open3d as o3d
import os
import copy

In [7]:
from src.preprocess import preprocess_point_cloud, prepare_clouds_scale_to_m
from src.metric import registration_metrics, symmetric_chamfer
from src.utils import load_landmarks, compute_rigid_transform, global_registration_ransac, refine_registration_icp, neighborhood_stats
from src.descriptor import build_spinnet_model, spinnet_features_for_pcd_profiled
from src.sweep import run_sweep

ImportError: cannot import name 'preprocess_point_cloud' from 'src.utils' (/Users/tuboshu/Documents/2025/M1/PRAT/Code/src/utils.py)

### Manual Annotation Registration (Dragon Tete D)

In [6]:
ref_pcd_path = "../../Data/ICP_test/ref_Tete_D.ply"
mov_pcd_path = "../../Data/ICP_test/move_Dragon_01_Transform.ply"
ref_lm_path  = "../../Data/ICP_test/picking_list_Tete_D.txt"
mov_lm_path  = "../../Data/ICP_test/picking_list_Dragon_01_Transform.txt"

# Read Point Cloud
ref_pcd = o3d.io.read_point_cloud(ref_pcd_path)
mov_pcd_raw = o3d.io.read_point_cloud(mov_pcd_path)

print(ref_pcd)
print(mov_pcd_raw)

# Read landmarks
Q = load_landmarks(ref_lm_path)  # reference
P = load_landmarks(mov_lm_path)  # moving

print("Landmark pairs:", P.shape)

T_init = compute_rigid_transform(P, Q)
print("Initial transform (from landmarks):\n", T_init)
# np.savetxt("T_landmark_4x4.txt", T_init)

# mov_pcd_landmark = mov_pcd.transform(T_init.copy())
# o3d.io.write_point_cloud("../Data/ICP_test/dragon_after_landmark.ply", mov_pcd_landmark)

mov_after_landmark = copy.deepcopy(mov_pcd_raw)
mov_after_landmark.transform(T_init)
#o3d.io.write_point_cloud("../../Data/ICP_test/dragon_after_landmark.ply", mov_after_landmark)


PointCloud with 1349372 points.
PointCloud with 1711170 points.
Landmark pairs: (5, 3)
Initial transform (from landmarks):
 [[ 9.62467722e-01  2.70397248e-01 -2.32639644e-02 -3.62131506e+01]
 [-2.70754231e-01  9.50764689e-01 -1.50793408e-01 -8.51340723e+02]
 [-1.86555667e-02  1.51432605e-01  9.88291524e-01 -2.97968502e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


PointCloud with 1711170 points.

In [17]:
max_corr_dist = 10.0

result_icp = o3d.pipelines.registration.registration_icp(
    mov_pcd_raw,                   # source
    ref_pcd,                       # target
    max_corr_dist,
    T_init,                        # init_trans from Landmark coarse registration
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(
        max_iteration=100
    )
)

print("ICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)


ICP result:
  fitness: 0.8167061133610337
  inlier_rmse: 1.7607851593499075
  T_icp:
 [[ 9.54561581e-01  2.96914653e-01 -2.55710273e-02 -4.80688633e+01]
 [-2.97020892e-01  9.40874457e-01 -1.62892129e-01 -9.31408116e+02]
 [-2.43059335e-02  1.63085697e-01  9.86312464e-01 -4.09150237e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [18]:
T_total = result_icp.transformation
np.savetxt("T_icp_total_4x4.txt", T_total)

mov_after_icp = copy.deepcopy(mov_pcd_raw)
mov_after_icp.transform(T_total)
o3d.io.write_point_cloud("../Data/ICP_test/dragon_after_icp_python.ply", mov_after_icp)


print("Saved aligned point cloud to dragon_after_icp_python.ply")
print("Saved ICP transform to T_icp_4x4.txt")


Saved aligned point cloud to dragon_after_icp_python.ply
Saved ICP transform to T_icp_4x4.txt


### FPFH + RANSAC Registration (Dragon Tete D)

In [66]:

ref_path = "../../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../../Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)

print("Loaded:")
print("  ref:", ref_pcd)
print("  mov:", mov_pcd)


voxel_size = 5.0

# 1) Preprocess for FPFH + RANSAC on downsampled point clouds

ref_down, ref_fpfh = preprocess_point_cloud(ref_pcd, voxel_size)
mov_down, mov_fpfh = preprocess_point_cloud(mov_pcd, voxel_size)

print("Downsampled:")
print("  ref_down:", ref_down)
print("  mov_down:", mov_down)
print("FPFH dims:", ref_fpfh.data.shape, mov_fpfh.data.shape)  # (33, N)


# 2) Global registration (RANSAC)
result_ransac = global_registration_ransac(mov_down, ref_down, mov_fpfh, ref_fpfh, voxel_size)
print("\nRANSAC result:")
print("  fitness:", result_ransac.fitness)
print("  inlier_rmse:", result_ransac.inlier_rmse)
print("  T_ransac:\n", result_ransac.transformation)

# 3) ICP refinement 
result_icp = refine_registration_icp(mov_pcd, ref_pcd, result_ransac.transformation, voxel_size, use_point_to_plane=True)
print("\nICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)


Loaded:
  ref: PointCloud with 1349372 points.
  mov: PointCloud with 1711170 points.
Downsampled:
  ref_down: PointCloud with 57450 points.
  mov_down: PointCloud with 59566 points.
FPFH dims: (33, 57450) (33, 59566)


In [8]:
# 4) Save transformed moving point cloud (optional)
mov_aligned = copy.deepcopy(mov_pcd)
mov_aligned.transform(result_icp.transformation)
o3d.io.write_point_cloud("../../Data/FPFH/mov_aligned_by_fpfh_ransac_icp_p2p.ply", mov_aligned)
print("\nSaved:", "mov_aligned_by_fpfh_ransac_icp_p2p.ply")


Saved: mov_aligned_by_fpfh_ransac_icp_p2p.ply


### Check with unity meter

In [72]:
ref_path = "Data/ICP_test/ref_Tete_D.ply"
mov_path = "Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)

print("Loaded:")
print("  ref:", ref_pcd)
print("  mov:", mov_pcd)


voxel_size = 0.005

ref_full, mov_full, ref_support, mov_support = prepare_clouds_scale_to_m(
        ref_path, mov_path, 1e-3, 0.005)

# 1) Preprocess for FPFH + RANSAC on downsampled point clouds

ref_down, ref_fpfh = preprocess_point_cloud(ref_full, voxel_size)
mov_down, mov_fpfh = preprocess_point_cloud(mov_full, voxel_size)

print("Downsampled:")
print("  ref_down:", ref_down)
print("  mov_down:", mov_down)
print("FPFH dims:", ref_fpfh.data.shape, mov_fpfh.data.shape)  # (33, N)

# 2) Global registration (RANSAC)
result_ransac = global_registration_ransac(mov_down, ref_down, mov_fpfh, ref_fpfh, voxel_size)
print("\nRANSAC result:")
print("  fitness:", result_ransac.fitness)
print("  inlier_rmse:", result_ransac.inlier_rmse)
print("  T_ransac:\n", result_ransac.transformation)

print("init_T translation:", result_ransac.transformation[:3,3])
print("ref_full diag:", np.linalg.norm(ref_full.get_axis_aligned_bounding_box().get_extent()))
print("mov_full diag:", np.linalg.norm(mov_full.get_axis_aligned_bounding_box().get_extent()))
print("ref_full center:", ref_full.get_center())
print("mov_full center:", mov_full.get_center())

# 3) ICP refinement on full-resolution clouds (or you can use downsampled first)
result_icp = refine_registration_icp(
    mov_full, ref_full,
    result_ransac.transformation,
    voxel_size,
    use_point_to_plane=True
)
print("\nICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)

Loaded:
  ref: PointCloud with 1349372 points.
  mov: PointCloud with 1711170 points.
Downsampled:
  ref_down: PointCloud with 57450 points.
  mov_down: PointCloud with 59565 points.
FPFH dims: (33, 57450) (33, 59565)

RANSAC result:
  fitness: 0.7834466549147989
  inlier_rmse: 0.0028374658134148792
  T_ransac:
 [[ 0.95450323  0.29707459 -0.02588963 -0.0492163 ]
 [-0.29721961  0.94073464 -0.16333655 -0.93242842]
 [-0.02416787  0.16360017  0.98623065 -0.03930241]
 [ 0.          0.          0.          1.        ]]
init_T translation: [-0.0492163  -0.93242842 -0.03930241]
ref_full diag: 1.331735633805758
mov_full diag: 1.364668405400209
ref_full center: [-0.70876686 -0.25534312 -4.61519417]
mov_full center: [-0.71520521 -0.28435465 -4.60412256]

ICP result:
  fitness: 0.7907022680388273
  inlier_rmse: 0.0012211446536510445
  T_icp:
 [[ 0.95432939  0.29766852 -0.02547288 -0.04748758]
 [-0.29774502  0.94062514 -0.16300995 -0.93248253]
 [-0.02456249  0.16314961  0.98629554 -0.04115251]
 [ 0

In [10]:
ref_path = "../../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../../Data/ICP_test/move_Dragon_01_Transform.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("Original not registered evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

Original not registered evaluation
{'mean': 28.790589237437796, 'std': 21.611957785388448, 'median': 24.060411166186718, 'p50': 24.060411166186718, 'p90': 61.29547679266786, 'p95': 71.37153082536858, 'coverage@5.0': 0.1196292595124973, 'trimmed_mean@5.0': 2.4777668386423106, 'trimmed_rmse@5.0': 2.83796814333311, 'coverage@10.0': 0.22594189940216344, 'trimmed_mean@10.0': 4.844387508719292, 'trimmed_rmse@10.0': 5.637451842177725}
Symmetric Chamfer: 54.16179091590028


In [19]:
ref_path = "../../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../../Data/ICP_test/dragon_after_icp_python.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("Manual landmark + ICP evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

Manual landmark + ICP evaluation
{'mean': 11.900873394379946, 'std': 26.68532721206653, 'median': 0.8916812282821849, 'p50': 0.8916812282821849, 'p90': 52.385716913856236, 'p95': 79.81466731250123, 'coverage@5.0': 0.7907560324222608, 'trimmed_mean@5.0': 0.9568073840270434, 'trimmed_rmse@5.0': 1.224656404608553, 'coverage@10.0': 0.8167061133610337, 'trimmed_mean@10.0': 1.1507352380483076, 'trimmed_rmse@10.0': 1.7607851593499064}
Symmetric Chamfer: 16.668003987122248


In [12]:
ref_path = "../../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../../Data/FPFH/mov_aligned_by_fpfh_ransac_icp_p2p.ply"
src = o3d.io.read_point_cloud(mov_path)
tgt = o3d.io.read_point_cloud(ref_path)

# src_aligned = copy.deepcopy(src)
# src_aligned.transform(T_total)

metrics = registration_metrics(src, tgt, thresholds=(5.0, 10.0))
print("FPFH + RANSAC + ICP evaluation")
print(metrics)

print("Symmetric Chamfer:", symmetric_chamfer(src, tgt))

FPFH + RANSAC + ICP evaluation
{'mean': 11.919651183532961, 'std': 26.723596740054912, 'median': 0.8878449859671382, 'p50': 0.8878449859671382, 'p90': 52.503750839545056, 'p95': 79.93291604220614, 'coverage@5.0': 0.7907022680388273, 'trimmed_mean@5.0': 0.953115089069285, 'trimmed_rmse@5.0': 1.2211446515516156, 'coverage@10.0': 0.8165477421880936, 'trimmed_mean@10.0': 1.1466224471712876, 'trimmed_rmse@10.0': 1.7575508328904914}
Symmetric Chamfer: 16.678766929614426


### SpinNet + RANSAC Registration (Dragon Tete D)

In [11]:
ref_path = "../../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../../Data/ICP_test/move_Dragon_01_Transform.ply"

ref_pcd = o3d.io.read_point_cloud(ref_path)
mov_pcd = o3d.io.read_point_cloud(mov_path)


bbox = ref_pcd.get_axis_aligned_bounding_box()
diag = np.linalg.norm(bbox.get_extent())
print("diag =", diag) # Verify the len unity

# 0) Unity transfer
scale = 1e-3
ref_pcd.scale(scale, center=ref_pcd.get_center())
mov_pcd.scale(scale, center=mov_pcd.get_center())

# 1) downsample（unity：m）
voxel_size = 0.005  # 5mm = 0.005m
ref_down = ref_pcd.voxel_down_sample(voxel_size)
mov_down = mov_pcd.voxel_down_sample(voxel_size)

ref_down, _ = ref_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
mov_down, _ = mov_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

bbox = ref_pcd.get_axis_aligned_bounding_box()
diag = np.linalg.norm(bbox.get_extent())
print("After scale, diag =", diag) # Verify the len unity

print("ref_down points:", np.asarray(ref_down.points).shape[0])
print("mov_down points:", np.asarray(mov_down.points).shape[0])

diag = 1331.7356338057582
After scale, diag = 1.3317356338051947


In [15]:
# 2) Build SpinNet model
ckpt_path = "SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

/tmp/ipykernel_3900923/2697050678.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


In [24]:
# 2.5)
print("Loaded ")
print("  ref_down:", ref_down)
print("  mov_down:", mov_down)

neighborhood_stats(ref_down, radius=0.30)

Loaded 
  ref_down: PointCloud with 57451 points.
  mov_down: PointCloud with 59565 points.
[radius=0.3] checked 5000 points in 10.37s
  nn count: min / median / p90 / max = 4800 24746.5 32293.500000000007 36282


array([32898, 23067, 28527, ..., 32842, 23483, 26997])

In [27]:
# 2.5) Time test
idx = np.random.choice(len(ref_down.points), 2000, replace=False)
ref_sub = ref_down.select_by_index(idx)

_, prof_sub = spinnet_features_for_pcd_profiled(
    ref_sub, model, patch_radius=0.30, N=2048, batch_size=32, device="cuda:0"
)

est_total = prof_sub["per_point_ms"] * len(ref_down.points) / 1000.0
print("estimated total seconds on full ref_down:", est_total)

estimated total seconds on full ref_down: 212.8129515532977


In [28]:
print(prof_sub)

{'num_pts': 2000, 'total_s': 7.408502952195704, 'per_point_ms': 3.704251476097852, 'kdtree_s': 0.1374448137357831, 'patch_build_s': 0.21178276371210814, 'stack_s': 0.019267170690000057, 'forward_s': 7.004434240050614, 'kdtree_ms_per_pt': 0.06872240686789155, 'patch_ms_per_pt': 0.10589138185605407, 'stack_ms_per_pt': 0.009633585345000029, 'fwd_ms_per_pt': 3.502217120025307}


In [30]:
# 3) SpinNet feature（patch_radius：m）
ref_feat, ref_prof = spinnet_features_for_pcd_profiled(ref_down, model, patch_radius=0.30, N=2048, batch_size=48, device="cuda:3")


#ref_feat = spinnet_features_for_pcd(ref_down, model, patch_radius=0.30, N=2048, batch_size=48, device="cuda:0")
#mov_feat = spinnet_features_for_pcd(mov_down, model, patch_radius=0.30, N=2048, batch_size=48, device="cuda:0")



In [32]:
print(ref_prof)

{'num_pts': 57451, 'total_s': 366.79874674696475, 'per_point_ms': 6.3845493855105175, 'kdtree_s': 116.37929208111018, 'patch_build_s': 55.9014504281804, 'stack_s': 0.3908678153529763, 'forward_s': 193.26412159670144, 'kdtree_ms_per_pt': 2.0257139489497167, 'patch_ms_per_pt': 0.9730283272385232, 'stack_ms_per_pt': 0.00680349890085423, 'fwd_ms_per_pt': 3.363981855784955}


In [35]:
mov_feat, mov_prof = spinnet_features_for_pcd_profiled(mov_down, model, patch_radius=0.30, N=2048, batch_size=48, device="cuda:3")

In [36]:
print(mov_prof)

{'num_pts': 59565, 'total_s': 353.89798430912197, 'per_point_ms': 5.941374705097322, 'kdtree_s': 107.84113709721714, 'patch_build_s': 52.47581277042627, 'stack_s': 0.3811010494828224, 'forward_s': 192.4380912426859, 'kdtree_ms_per_pt': 1.8104782522826683, 'patch_ms_per_pt': 0.8809840136057462, 'stack_ms_per_pt': 0.006398070166756021, 'fwd_ms_per_pt': 3.23072427168112}


In [40]:
# 4) RANSAC 
result_ransac = global_registration_ransac(mov_down, ref_down, mov_feat, ref_feat, voxel_size)
print("\nRANSAC result:")
print("  fitness:", result_ransac.fitness)
print("  inlier_rmse:", result_ransac.inlier_rmse)
print("  T_ransac:\n", result_ransac.transformation)



[Open3D WARNING] Too few correspondences (514) after mutual filter, fall back to original correspondences.

RANSAC result:
  fitness: 0.6604213883992277
  inlier_rmse: 0.004330643965148919
  T_ransac:
 [[ 9.43375853e-01  3.31017676e-01 -2.16632845e-02 -3.96642037e+01]
 [-3.29924565e-01  9.29454455e-01 -1.65118739e-01 -9.87221529e+02]
 [-3.45221849e-02  1.62916281e-01  9.86035752e-01 -5.37294406e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [41]:
# 5) ICP
result_icp = refine_registration_icp(mov_pcd, ref_pcd, result_ransac.transformation, voxel_size, use_point_to_plane=False)
print("\nICP result:")
print("  fitness:", result_icp.fitness)
print("  inlier_rmse:", result_icp.inlier_rmse)
print("  T_icp:\n", result_icp.transformation)



ICP result:
  fitness: 0.7907040212252435
  inlier_rmse: 0.0012213668000526324
  T_icp:
 [[ 9.54331532e-01  2.97661486e-01 -2.54748046e-02 -5.88602838e+01]
 [-2.97738628e-01  9.40628721e-01 -1.63000974e-01 -9.51272785e+02]
 [-2.45567794e-02  1.63141803e-01  9.86296972e-01 -4.53309096e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [42]:
# 4) Save transformed moving point cloud (optional)
mov_aligned = copy.deepcopy(mov_pcd)
mov_aligned.transform(result_icp.transformation)
o3d.io.write_point_cloud("../../Data/SpinNet/mov_aligned_by_SpinNet_ransac_icp_p2p.ply", mov_aligned)
print("\nSaved:", "mov_aligned_by_SpinNet_ransac_icp_p2p.ply")


Saved: mov_aligned_by_SpinNet_ransac_icp_p2p.ply


### Training SpinNet
load training files ../../data/3DMatch/patches/train/train_anc&pos_20_2048_2000.pkl
Epoch 19: Loss 1.0435524266898795, time 10648.4401s
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_4000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_6000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_8000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_10000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_12000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_14000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_16000.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_17043.pkl
load training files ../../data/3DMatch/patches/val/val_anc&pos_2_2048_2000.pkl
Evaluation: Epoch 19: Loss 1.2269546277540968
Avg one epoch time: 10514.17, total 20 epochs time: 214677.52
Training finish!... save training results

### Sweep run seeds, comparing FPFH vs SpinNet on different downsample ratios

###  Tete_D Comparison

In [34]:
ckpt_path = "../model/SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

ref_path = "../../Data/ICP_test/ref_Tete_D.ply"
mov_path = "../../Data/ICP_test/move_Dragon_01_Transform.ply"
results = run_sweep(ref_path, mov_path, model, device="cuda:3", batch_size=96)

/tmp/ipykernel_3900923/2825193392.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


Prepared:
  ref_full diag: 1.331735633805758 center: [-0.70876686 -0.25534312 -4.61519417]
  mov_full diag: 1.364668405400209 center: [-0.71520521 -0.28435465 -4.60412256]
  ref_support n: 57450
  mov_support n: 59565
  K_base=min_support, n_base=57450
  Ks: [2872, 5745, 11490, 22980] from fracs: (0.05, 0.1, 0.2, 0.4)

=== Run K=2872 (5.0% of base), seed=0 ===


/users/nfs/Etu0/21400500/PRAT/SpinNet/script/common.py:559: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  c = torch.cross(a, b)


[Open3D WARNING] Too few correspondences (155) after mutual filter, fall back to original correspondences.
[Saved] Data/Registrations/K_2872/seed_0/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_2872/seed_0/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/Tete_D/K_2872/seed_0/out.json
  FPFH total_s: 6.759746282361448 cov@5mm: 0.7907022680388273 rmse@5mm: 0.0012211446547372403
  Spin total_s: 41.73403585422784 cov@5mm: 0.7907022680388273 rmse@5mm: 0.0012211446577079642

=== Run K=2872 (5.0% of base), seed=1 ===
[Open3D WARNING] Too few correspondences (171) after mutual filter, fall back to original correspondences.
[Saved] Data/Registrations/K_2872/seed_1/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_2872/seed_1/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/Tete_D/K_2872/seed_1/out.json
  FPFH total_s: 6.432646093890071 cov@5mm: 0.7907022680388273 rmse@5mm: 0.0012211446554003529
  Spin total_s: 40.625422425568104 cov@5mm: 0.7907022680388273 rmse@5mm: 0.001221144655389953

In [35]:
print(results)

[{'K': 2872, 'seed': 0, 'voxel_size': 0.005, 'patch_radius': 0.3, 'ref_support_n': 57450, 'mov_support_n': 59565, 'ref_kp_n': 2872, 'mov_kp_n': 2872, 'fpfh_feat_s': 0.38458248507231474, 'fpfh_ransac_s': 0.24954984430223703, 'fpfh_icp_s': 6.125613952986896, 'fpfh_total_s': 6.759746282361448, 'fpfh_ransac_fitness': 0.2862116991643454, 'fpfh_ransac_rmse': 0.005325186560449784, 'fpfh_icp_fitness': 0.7907022680388273, 'fpfh_icp_rmse': 0.0012211446547372418, 'fpfh_cov_5mm': 0.7907022680388273, 'fpfh_cov_10mm': 0.8165477421880936, 'fpfh_trimrmse_5mm': 0.0012211446547372403, 'fpfh_trimrmse_10mm': 0.0017575509092742716, 'fpfh_chamfer': 0.01667876527225759, 'spinnet_ref_feat_s': 17.415413820184767, 'spinnet_mov_feat_s': 16.933637131005526, 'spinnet_feat_s': 34.34905095119029, 'spinnet_ransac_s': 0.4364918367937207, 'spinnet_icp_s': 6.948493066243827, 'spinnet_total_s': 41.73403585422784, 'spinnet_ransac_fitness': 0.21622562674094709, 'spinnet_ransac_rmse': 0.005466246926013297, 'spinnet_icp_fitn

In [52]:
ckpt_path = "../model/SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

ref_path = "../../Data/3D/B1G_Complet.obj"
mov_path = "../../Data/2D/mov/B1G_10M_Transform.obj"
results = run_sweep(ref_path, mov_path, model, device="cuda:3", batch_size=96, save_dir="B1G")

/tmp/ipykernel_3900923/2825193392.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


Prepared:
  ref_full diag: 3.065587826340805 center: [-0.03474549 -0.05078032 -0.62796102]
  mov_full diag: 3.0421886011408157 center: [-0.493421   -0.31135135  1.04459209]
  ref_support n: 451634
  mov_support n: 448032
  K_base=min_support, n_base=448032
  Ks: [22402, 44803, 89606, 179213] from fracs: (0.05, 0.1, 0.2, 0.4)

=== Run K=22402 (5.0% of base), seed=0 ===
[Saved] Data/Registrations/K_22402/seed_0/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_22402/seed_0/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/B1G/K_22402/seed_0/out.json
  FPFH total_s: 29.873449128121138 cov@5mm: 0.0014692894484199333 rmse@5mm: 0.0031377315107188114
  Spin total_s: 392.66273540630937 cov@5mm: 0.889174739751606 rmse@5mm: 0.002166099135021695

=== Run K=22402 (5.0% of base), seed=1 ===
[Saved] Data/Registrations/K_22402/seed_1/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_22402/seed_1/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/B1G/K_22402/seed_1/out.json
  FPFH total_s: 29.3763708

In [53]:
print(results)

[{'K': 22402, 'seed': 0, 'voxel_size': 0.005, 'patch_radius': 0.3, 'ref_support_n': 451634, 'mov_support_n': 448032, 'ref_kp_n': 22402, 'mov_kp_n': 22402, 'fpfh_feat_s': 3.6547739319503307, 'fpfh_ransac_s': 5.723358812741935, 'fpfh_icp_s': 20.49531638342887, 'fpfh_total_s': 29.873449128121138, 'fpfh_ransac_fitness': 0.0, 'fpfh_ransac_rmse': 0.0, 'fpfh_icp_fitness': 0.0014692894484199333, 'fpfh_icp_rmse': 0.003137731510718808, 'fpfh_cov_5mm': 0.0014692894484199333, 'fpfh_cov_10mm': 0.0026431196017625068, 'fpfh_trimrmse_5mm': 0.0031377315107188114, 'fpfh_trimrmse_10mm': 0.005530341803614228, 'fpfh_chamfer': 2.0489803121514667, 'spinnet_ref_feat_s': 200.80061556305736, 'spinnet_mov_feat_s': 177.4405180066824, 'spinnet_feat_s': 378.24113356973976, 'spinnet_ransac_s': 9.937998907640576, 'spinnet_icp_s': 4.483602928929031, 'spinnet_total_s': 392.66273540630937, 'spinnet_ransac_fitness': 0.2678332291759664, 'spinnet_ransac_rmse': 0.0053484986860953335, 'spinnet_icp_fitness': 0.889174739751606

In [54]:
ckpt_path = "SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

ref_path = "Data/3D/B2G.obj"
mov_path = "Data/2D/mov/B2G_10M_Transform.obj"
results = run_sweep(ref_path, mov_path, model, K_fracs=(0.05, 0.10, 0.20), device="cuda:3", batch_size=96, save_dir="B2G")

/tmp/ipykernel_3900923/2825193392.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


Prepared:
  ref_full diag: 3.582339289619445 center: [-0.04367127 -0.0283486  -0.53564847]
  mov_full diag: 3.6090151904151897 center: [-0.46008407 -0.2810625  -2.15677382]
  ref_support n: 200112
  mov_support n: 457980
  K_base=min_support, n_base=200112
  Ks: [10006, 20011, 40022] from fracs: (0.05, 0.1, 0.2)

=== Run K=10006 (5.0% of base), seed=0 ===
[Saved] Data/Registrations/K_10006/seed_0/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_10006/seed_0/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/B2G/K_10006/seed_0/out.json
  FPFH total_s: 10.606607121415436 cov@5mm: 0.0012199443705367035 rmse@5mm: 0.003561943380193059
  Spin total_s: 129.96690933126956 cov@5mm: 0.45257696249051044 rmse@5mm: 0.0034159685318882233

=== Run K=10006 (5.0% of base), seed=1 ===
[Saved] Data/Registrations/K_10006/seed_1/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_10006/seed_1/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/B2G/K_10006/seed_1/out.json
  FPFH total_s: 10.640232373028994 co

In [55]:
print(results)

[{'K': 10006, 'seed': 0, 'voxel_size': 0.005, 'patch_radius': 0.3, 'ref_support_n': 200112, 'mov_support_n': 457980, 'ref_kp_n': 10006, 'mov_kp_n': 10006, 'fpfh_feat_s': 2.2986414963379502, 'fpfh_ransac_s': 1.8749016225337982, 'fpfh_icp_s': 6.433064002543688, 'fpfh_total_s': 10.606607121415436, 'fpfh_ransac_fitness': 0.0, 'fpfh_ransac_rmse': 0.0, 'fpfh_icp_fitness': 0.0012199443705367035, 'fpfh_icp_rmse': 0.003561943380193053, 'fpfh_cov_5mm': 0.0012199443705367035, 'fpfh_cov_10mm': 0.0033374478123797554, 'fpfh_trimrmse_5mm': 0.003561943380193059, 'fpfh_trimrmse_10mm': 0.006420014478097012, 'fpfh_chamfer': 2.0749589498997993, 'spinnet_ref_feat_s': 49.94044124241918, 'spinnet_mov_feat_s': 73.88352590240538, 'spinnet_feat_s': 123.82396714482456, 'spinnet_ransac_s': 2.9369815196841955, 'spinnet_icp_s': 3.2059606667608023, 'spinnet_total_s': 129.96690933126956, 'spinnet_ransac_fitness': 0.08974615230861484, 'spinnet_ransac_rmse': 0.005681222542263653, 'spinnet_icp_fitness': 0.45257696249051

In [ ]:
ckpt_path = "SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

ref_path = "Data/3D/B2G.obj"
mov_path = "Data/2D/mov/B2G_10M_Transform.obj"
results = run_sweep(ref_path, mov_path, model, K_fracs=[0.40], device="cuda:3", batch_size=96, save_dir="B2G")

/tmp/ipykernel_3900923/2825193392.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


Prepared:
  ref_full diag: 3.582339289619445 center: [-0.04367127 -0.0283486  -0.53564847]
  mov_full diag: 3.6090151904151897 center: [-0.46008407 -0.2810625  -2.15677382]
  ref_support n: 200112
  mov_support n: 457980
  K_base=min_support, n_base=200112
  Ks: [80045] from fracs: [0.4]

=== Run K=80045 (40.0% of base), seed=0 ===


In [ ]:
print(results)

In [8]:
ckpt_path = "SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

ref_path = "Data/3D/Tete_D.obj"
mov_path = "Data/2D/mov/Dragon_01_Transform.obj"
results = run_sweep(ref_path, mov_path, model, K_fracs=(0.05, 0.10, 0.20, 0.40), device="cuda:3", batch_size=96, save_dir="Tete_D_new")

/tmp/ipykernel_303690/3660043762.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


Prepared:
  ref_full diag: 1.3069279333879589 center: [ 0.00150416 -0.03673088 -0.25881325]
  mov_full diag: 1.3646683957015815 center: [-0.42871775 -0.44888654 -4.54728858]
  ref_support n: 52425
  mov_support n: 59566
  K_base=min_support, n_base=52425
  Ks: [2621, 5242, 10485, 20970] from fracs: (0.05, 0.1, 0.2, 0.4)

=== Run K=2621 (5.0% of base), seed=0 ===


/users/nfs/Etu0/21400500/PRAT/SpinNet/script/common.py:559: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  c = torch.cross(a, b)


[Open3D WARNING] Too few correspondences (182) after mutual filter, fall back to original correspondences.
[Saved] Data/Registrations/K_2621/seed_0/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_2621/seed_0/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/Tete_D_new/K_2621/seed_0/out.json
  FPFH total_s: 13.257196213118732 cov@5mm: 0.029299251389400236 rmse@5mm: 0.002818263297820979
  Spin total_s: 36.25657084956765 cov@5mm: 0.7907028524342994 rmse@5mm: 0.001221151851626712

=== Run K=2621 (5.0% of base), seed=1 ===
[Open3D WARNING] Too few correspondences (187) after mutual filter, fall back to original correspondences.
[Saved] Data/Registrations/K_2621/seed_1/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_2621/seed_1/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/Tete_D_new/K_2621/seed_1/out.json
  FPFH total_s: 6.312729398719966 cov@5mm: 0.0 rmse@5mm: nan
  Spin total_s: 36.64664318691939 cov@5mm: 0.7907028524342994 rmse@5mm: 0.0012211518320016882

=== Run K=2621 (5.0% o

In [9]:
print(results)

[{'K': 2621, 'seed': 0, 'voxel_size': 0.005, 'patch_radius': 0.3, 'ref_support_n': 52425, 'mov_support_n': 59566, 'ref_kp_n': 2621, 'mov_kp_n': 2621, 'fpfh_feat_s': 0.39557130821049213, 'fpfh_ransac_s': 0.24434556812047958, 'fpfh_icp_s': 12.61727933678776, 'fpfh_total_s': 13.257196213118732, 'fpfh_ransac_fitness': 0.013735215566577643, 'fpfh_ransac_rmse': 0.005663828920853665, 'fpfh_icp_fitness': 0.029299251389400236, 'fpfh_icp_rmse': 0.0028182632978209733, 'fpfh_cov_5mm': 0.029299251389400236, 'fpfh_cov_10mm': 0.05220054115020717, 'fpfh_trimrmse_5mm': 0.002818263297820979, 'fpfh_trimrmse_10mm': 0.005438841622193306, 'fpfh_chamfer': 0.4894981210140766, 'spinnet_ref_feat_s': 15.024606600403786, 'spinnet_mov_feat_s': 15.20520151592791, 'spinnet_feat_s': 30.229808116331697, 'spinnet_ransac_s': 0.37912500835955143, 'spinnet_icp_s': 5.647637724876404, 'spinnet_total_s': 36.25657084956765, 'spinnet_ransac_fitness': 0.18237314002289204, 'spinnet_ransac_rmse': 0.00566679040600737, 'spinnet_icp

In [ ]:


ckpt_path = "SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

ref_path = "Data/3D/Poutre_02_2.obj"
mov_path = "Data/2D/mov/B3D_10M_Transform.obj"
results = run_sweep(ref_path, mov_path, model, K_fracs=(0.05, 0.10, 0.20, 0.40), device="cuda:3", batch_size=96, save_dir="Poutre_02")

/tmp/ipykernel_303690/3660043762.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


Prepared:
  ref_full diag: 4.155313363964906 center: [ 0.00267435 -0.02397745 -0.63449418]
  mov_full diag: 3.183803198271518 center: [-0.45578293 -0.24716486 -4.88637701]
  ref_support n: 213157
  mov_support n: 400580
  K_base=min_support, n_base=213157
  Ks: [10658, 21316, 42631, 85263] from fracs: (0.05, 0.1, 0.2, 0.4)

=== Run K=10658 (5.0% of base), seed=0 ===
[Open3D WARNING] Too few correspondences (456) after mutual filter, fall back to original correspondences.
[Saved] Data/Registrations/K_10658/seed_0/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_10658/seed_0/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/Poutre_02/K_10658/seed_0/out.json
  FPFH total_s: 5.550062914378941 cov@5mm: 0.0 rmse@5mm: nan
  Spin total_s: 133.785623062402 cov@5mm: 0.0 rmse@5mm: nan

=== Run K=10658 (5.0% of base), seed=1 ===
[Open3D WARNING] Too few correspondences (432) after mutual filter, fall back to original correspondences.
[Saved] Data/Registrations/K_10658/seed_1/mov_aligned_FPFH.ply

In [ ]:
print(results)

In [12]:
ckpt_path = "SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

ref_path = "Data/3D/B2G.obj"
mov_path = "Data/2D/mov/B2G_10M_Transform.obj"
results = run_sweep(ref_path, mov_path, model, K_fracs=[0.40], seeds=(2, 3, 4), device="cuda:3", batch_size=96, save_dir="B2G")

/tmp/ipykernel_303690/3660043762.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=map_location)


Prepared:
  ref_full diag: 3.582339289619445 center: [-0.04367127 -0.0283486  -0.53564847]
  mov_full diag: 3.6090151904151897 center: [-0.46008407 -0.2810625  -2.15677382]
  ref_support n: 200112
  mov_support n: 457980
  K_base=min_support, n_base=200112
  Ks: [80045] from fracs: [0.4]

=== Run K=80045 (40.0% of base), seed=2 ===
[Open3D WARNING] Too few correspondences (7300) after mutual filter, fall back to original correspondences.
[Saved] Data/Registrations/K_80045/seed_2/mov_aligned_FPFH.ply
[Saved] Data/Registrations/K_80045/seed_2/mov_aligned_SpinNet.ply
[Saved] Data/Registrations/B2G/K_80045/seed_2/out.json
  FPFH total_s: 134.6887363800779 cov@5mm: 0.0012199443705367035 rmse@5mm: 0.003561943380193056
  Spin total_s: 1209.2349676080048 cov@5mm: 0.45046905861092734 rmse@5mm: 0.003443395302146901

=== Run K=80045 (40.0% of base), seed=3 ===
[Open3D WARNING] Too few correspondences (7234) after mutual filter, fall back to original correspondences.
[Saved] Data/Registrations/K_8

In [13]:
import torch
torch.cuda.empty_cache()

### Test

In [ ]:
ckpt_path = "../model/SpinNet/pre-trained_models/3DMatch_best.pkl"   
model = build_spinnet_model(ckpt_path,
                            des_r=0.30, rad_n=9, azi_n=80, ele_n=40,
                            voxel_r=0.04, voxel_sample=30,
                            dataset="3DMatch",
                            device="cuda:3")

ref_path = "../../Data/3D/B1G_Complet.obj"
mov_path = "../../Data/2D/mov/B1G_10M_Transform.obj"
results = run_sweep(ref_path, mov_path, model, device="cuda:3", batch_size=96, save_dir="B1G")